# Spin Analyzer Lab (Stern-Gerlach Experiment)

Some basic functions and definitions... nothing to see here. 

In [1]:
from pylab import *

In [2]:
def newstate(n):
    #random coefficients
    v = zeros(n,dtype='complex')
    for i in range(0,n):
        r = rand()
        phi = 2*pi*rand()
        v[i] = r*exp(-1j*phi)

    v = v/sqrt(sum(abs(v[:])**2))
    
    return v

def measure_op(v,op,n):
    #operator
    w,u = eigh(op)
    
    #overlap coefficients
    #c = zeros(n,dtype='complex')
    
    for i in range(0,n):
        c = squeeze(asarray(dot(v,conj(u))))
        
    #measure by selecting random number
    r = rand()
    
    csum = abs(c[0])**2
    nf = 0
    
    while (r>csum):
        nf = nf + 1
        csum = csum + abs(c[nf])**2
        
    vf = squeeze(asarray(u[:,nf]))
    val = w[nf]
    
    return val, vf

For simplicity, we work in units where $\hbar = 1$. To convert to actual values, multiply all spin measurements by $\hbar$ in your units system of choice.

In [3]:
#operators
sz = 0.5*matrix([[1.0,0.0],[0.0,-1.0]])
sx = 0.5*matrix([[0.0,1.0],[1.0,0.0]])
sy = 0.5*matrix([[0.0,-1j],[1j,0.0]])

Initialize a random (unpolarized) state by calling <b>newstate(2)</b> and feed it into a vector <b>v</b>. This function should return the meausured value $\lambda$ and the state of the system <i>after</i> the measurement $\left|\psi\right\rangle$ represented by a vector $\vec{u}$ as $(\lambda, \vec{u})$.

Repeat several times to observe whether the measurement fluctuates in time.

In [4]:
v = newstate(2)

Examine the state by printing its components.

In [5]:
print(v)

[ 0.36407982+0.09417192j -0.03461199-0.92594792j]


Check normalization

In [6]:
sum(abs(v)**2)

1.0

Measure $S^{z}$ by calling <b>measure_op(v,sz,2)</b>

In [7]:
measure_op(v,sz,2)

(0.5, array([1., 0.]))

Imagine making many identical copies of <b>v</b> and measuring $\hat{S}^{z}$ on each one. Should we expect the same result each time? Execute the cell above until you feel sure about the answer. What happens to the state when the value changes?

Write a short loop to perform this measurement a number (>10) of times <i>on the same state</i> <b>v</b>. After each measurement, print the measured value.

Within the loop, when you csall <b>measure_op()</b>, you'll need to assign each of its outputs to a variable. The proper syntax is:

<b>val,state = measure_op(v,sz,2)</b>

In [8]:
for i in range(0,20):
    val,state = measure_op(v,sz,2)
    print(val)

0.5
-0.5
0.5
0.5
-0.5
0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
0.5
0.5
0.5
0.5
-0.5
-0.5
-0.5
-0.5


What is the <i>expectation value</i> of $\hat{S}^{z}$? That is, what is the <i>average</i> value $\left\langle \hat{S}^{z}\right\rangle$? Write a short loop to compute this. You should make sure that you have chosen enough samples for the result converge. Loosely, keep increasing the number of samples until the result doesn't change appreciably.

In [1]:
N = 10000
stot = 0

for i in range(0,N):
    s,state = measure_op(v,sz,2)
    stot = stot + s
    
szav = 0.5*(abs(v[0])**2 - abs(v[1])**2)
    
print('Measured <Sz> = ' + str(stot/N))
print('Measured <Sz> = ' + str(szav))

NameError: name 'measure_op' is not defined

Note that each instance of a measurement returns a <i>random</i> result. The probability of each possible outcome is weighted, but a given measurement's outcome is unpredictable except for very specific states with respect to a given operator (eigenstates). We'll see in class that from the state vector, you can directly compute this expectation value.

Calculate $0.5\times\left(|v[0]|^{2}-|v[1]|^{2}\right)$. How does it compare to your numerical estimate of the expectation value?

In [10]:
0.5*(abs(v[0])**2 - abs(v[1])**2)

-0.3585775328725139

If we let $v[0]\rightarrow\alpha$ and $v[1]\rightarrow\beta$, some magic formulas we'll derive later are:

$$ \begin{array}{ccc}\left\langle \hat{S}^{x}\right\rangle & = & \frac{1}{2}\left(\alpha^{*}\beta + \beta^{*}\alpha\right),\\
\left\langle \hat{S}^{y}\right\rangle & = & \frac{-i}{2}\left(\alpha^{*}\beta - \beta^{*}\alpha\right),\\
\left\langle \hat{S}^{z}\right\rangle & = & \frac{1}{2}\left(|\alpha|^{2} - |\beta|^{2}\right).\end{array}$$


Compute the two expressions you haven't already examined and calculate an average as before for each.

In [11]:
print(0.5*(conj(v[0])*v[1]+conj(v[1])*v[0]))
print((0.5/1j)*(conj(v[0])*v[1]-conj(v[1])*v[0]))

(-0.09979981921383171+0j)
(-0.3338594749351105-0j)


In [12]:
N = 10000
sxtot = 0
sytot = 0

for i in range(0,N):
    s1,state = measure_op(v,sx,2)
    sxtot = sxtot + s1
    
    s2,state = measure_op(v,sy,2)
    sytot = sytot + s2
    
print('<Sx> = ' + str(sxtot/N))
print('<Sy> = ' + str(sytot/N))

<Sx> = -0.0967
<Sy> = -0.3351


# 1. STERN-GERLACH EXPERIMENT (BASIC VERSION)

In the most basic form of the Stern-Gerlach experiment, we observe a basic component of "quantum weirdness." Imagine passing a beam of unpolarized spins through an analyzer is able to filter spin up $\left|+\right\rangle$ from spin down $\left|-\right\rangle$ along a given axis. This is the quantum analog of a simple polarizing film (like sunglasses). Since the beam is unpolarized, choosing any axis should filter out about half of the spins in each component.

To create such a beam, we'll need to initialize a large number of "random" spin states. Write a short script to create a large number of random states and measure $\hat{S}^{z}$. Keep track of how many of each state the measurement finds. For now you can discard the information about the state vectors produced by the measurement. Make sure the number of spins created is large enough to create a sensible distribution (note: about half of the intensity in a beam of unpolarized light will make it through a polarizer with any orientation).

In [13]:
N = 10000

Nup = 0
Ndown = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')

The beam produced 4867 |+> states and 5133 |-> states.


Copy the block of code and select the $\left|+\right\rangle$ spins for further analysis. Measure $\hat{S}^{z}$ <i>again</i> to confirm that these spins have a well-defined component of spin along this axis.

To accomplish this, you'll need to insert a new conditional after the spin is initially measured and feed the output state of the first measurement into the next call to the measurement function.

In [14]:
N = 10000

Nup = 0
Ndown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        s2,state2 = measure_op(state,sz,2)
        if s2>0:
            Nup2 = Nup2 + 1
        else:
            Ndown2 = Ndown2 + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('Of the ' + str(Nup) + ' |+> states, a second measurement revealed ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')

The beam produced 5061 |+> states and 4939 |-> states.
Of the 5061 |+> states, a second measurement revealed 5061 |+> states and 0 |-> states.


Now we are going to repeat the experiment but with an additional measurement. After selecting the $\left|+\right\rangle$ spins but <i>before</i> confirming the measurement of $\hat{S}^{z}$, we are going to measure $\hat{S}^{x}$ and select only those spins with $S^{x} = + \frac{1}{2}$ for further analysis. Classically, this is just a subset of the spins we have already analyzed (twice). To accomplish this, we will need to add yet another conditional statement and only select those, $\left|+\right\rangle_{x}$, for the second measurement of $S^{z}$. 

It is <b>critical</b> to note that we will feed the state returned by the measurement into the next measurement. It is probably simplest to convert the second measurement of $\hat{S}^{z}$ into a measurement of $\hat{S}^{x}$ and then insert an additional measurement of $\hat{S}^{z}$ for those states satisfying $S^{x}>0$ (all of which were previously selected for also having $S^{z}>0$. 


In [209]:
N = 10000

Nup = 0
Ndown = 0

Nxup = 0
Nxdown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        s2,state2 = measure_op(state,sx,2)
        if s2>0:
            Nxup = Nxup + 1
            s3,state3 = measure_op(state2,sz,2)
            if s3>0:
                Nup2 = Nup2 + 1
            else:
                Ndown2 = Ndown2 + 1
        else:
            Nxdown = Nxdown + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('Of the ' + str(Nxup) + ' states with Sx>0, a second measurement revealed ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')



The beam produced 5030 |+> states and 4970 |-> states.
Of the 2530 states with Sx>0, a second measurement revealed 1293 |+> states and 1237 |-> states.


If programed correctly, you should find a very curious result. The first "filtering" selected about half of the initial states which had $S^{z} = +\frac{1}{2}$. We then measured $\hat{S}^{x}$, finding approximately a 50/50 split (of the first half) between $S^{x} = \frac{1}{2}$ and $S^{x} = -\frac{1}{2}$. It should be <b>strongly emphasized</b> that the $z$ component for all of these spins was just measured and determined to be $\frac{1}{2}$.

Now taking only the subset with $S^{x}>0$ (about a quarter of the initial total), we again measure $S^{z}$. Just as you're probably wondering why we're going through all of this hassle to confirm (yet again) something we already "know," something interesting happens: only half the spins in this sample <i>still</i> have $S^{z}=\frac{1}{2}$. Somehow, the measurement of $\hat{S}^{x}$ changed the state of $\hat{S}^{z}$.

We only looked again at those (supposedly "up") spins returning $S^{x}>0$. What type of distribution in $S^{z}$ values would you expect to find for those with $S^{x}<0$? Copy the above block of code and make the appropriate changes to investigate the $z$ component of spins with $S^{x}<0$.

Subtle point (that you might or might not be thinking about): you'll have to <i>repeat</i> the experiment by creating a new set of initial spins. Maybe re-run the initial block enough times to convince yourself that this result is rigid (i.e., the distribution you found above doesn't change for another batch of unpolarized spins except for small statistical fluctuations).

In [212]:
N = 10000

Nup = 0
Ndown = 0

Nxup = 0
Nxdown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        s2,state2 = measure_op(state,sx,2)
        if s2>0:
            Nxup = Nxup + 1
        else:
            Nxdown = Nxdown + 1
            s3,state3 = measure_op(state2,sz,2)
            if s3>0:
                Nup2 = Nup2 + 1
            else:
                Ndown2 = Ndown2 + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('')
print('Of the ' + str(Nxup) + ' states with Sx<0, a second measurement revealed ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')



The beam produced 5045 |+> states and 4955 |-> states.

Of the 2516 states with Sx<0, a second measurement revealed 1249 |+> states and 1280 |-> states.


## Questions to answer: 

Make an initial guess (or state the answer, if you feel confident) and support your answer by modifying the simulation above to model each scenario.

(1) Suppose you began with 100000 unpolarized ("randomly oriented") spins. What fraction of the initial spins would return a measurement of $S^{z} = +\frac{1}{2}$ after the first measurement of $\hat{S}^{z}$?


<b>Answer:</b> 50000 (about half)

In [213]:
N = 100000

Nup = 0
Ndown = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')

The beam produced 50066 |+> states and 49934 |-> states.


(2) If $\hat{S}^{z}$ was measured for the $\left|+\right\rangle$ spins again, <i>immediately</i> after the first measurement, how many spins would have $S^{z}=+\frac{1}{2}$?


<b>Answer: </b> 50000 (same answer); note that since all spins are re-measured, you don't have to "filter" them after the first measurement. It would take more code, but you could also show (as we saw earlier) that each individual value is the same for both measurements. Some will be up and some will be down, but for a single spin, the second measurement matches the first measurement exactly.

In [215]:
N = 100000

Nup = 0
Ndown = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    s2,state2 = measure_op(state,sz,2)
    if s2>0:
        Nup = Nup + 1
    else:
        Ndown = Ndown +1
    
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')

The beam produced 50378 |+> states and 49622 |-> states.


(3) Suppose only a single measurement of $\hat{S}^{z}$ is performed, and only those spins with $S^{z} =-\frac{\hbar}{2}$ are selected for further analysis (how many spins does this leave?). The operator $\hat{S}^{x}$ is then measured, and only those spins with $S^{x} =\frac{1}{2}$ are selected. These spins are then passed through another analyzer which measures $\hat{S}^{z}$. How many spins (of the subset with $S^{x} = \frac{1}{2}$) now have $S^{z}= \frac{1}{2}$? $S^{z} = -\frac{1}{2}$?

<b>Answer: </b> The initial filtering selects about 50000 spins (about half of the initial batch). Filtering by the value of $S^{x}=\frac{1}{2}$ keeps about 25000 spins. Of those, about 12500 will still have $S^{z} = \frac{1}{2}$.

In [216]:
N = 100000

Nup = 0
Ndown = 0

Nxup = 0
Nxdown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        s2,state2 = measure_op(state,sx,2)
        if s2>0:
            Nxup = Nxup + 1
            s3,state3 = measure_op(state2,sz,2)
            if s3>0:
                Nup2 = Nup2 + 1
            else:
                Ndown2 = Ndown2 + 1
        else:
            Nxdown = Nxdown + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('Of the ' + str(Nxup) + ' states with Sx>0, a second measurement revealed ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')



The beam produced 49682 |+> states and 50318 |-> states.
Of the 24866 states with Sx>0, a second measurement revealed 12487 |+> states and 12379 |-> states.


(4) For the scenario considered in (3), if an additional analyzer was inserted which measured $\hat{S}^{x}$ a second time immediately after the first measurement, how would the number of spins with $S^{z}=\frac{1}{2}$ at the end of the path change? 

<b>Answer: </b> No change. Just like a second measurement of $\hat{S}^{z}$ immediately after the first simply returns the identical result, a second measurement of $\hat{S}^{x}$ <i>immediately after the first</i> will have no affect on the rest of the analyzer chain.

In [217]:
N = 100000

Nup = 0
Ndown = 0

Nxup = 0
Nxdown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        s2,state2 = measure_op(state,sx,2)
        s2a,state2a = measure_op(state2,sx,2) #<- second Sx measurement
        
        if s2a>0:
            Nxup = Nxup + 1                  #<- second measuremnt used
            s3,state3 = measure_op(state2a,sz,2) #<- output state of 2nd measurement used
            if s3>0:
                Nup2 = Nup2 + 1
            else:
                Ndown2 = Ndown2 + 1
        else:
            Nxdown = Nxdown + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('Of the ' + str(Nxup) + ' states with Sx>0, a second measurement revealed ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')


The beam produced 50203 |+> states and 49797 |-> states.
Of the 25306 states with Sx>0, a second measurement revealed 12669 |+> states and 12637 |-> states.


(5) For the scenario considered in (3), suppose those spins with $S^{x} = -\frac{1}{2}$ are also run through a separate analyzer which measures $\hat{S}^{z}$ a second time for this set of spins. This means, after initially selecting spins with $S^{z}$ from the unpolarized spins and separating those according to the value of $S^{x}$, all spins which initially had $S^{z} = \frac{1}{2}$ are analyzed with respect to $\hat{S}^{z}$ <i>again</i>. About how many will have $S^{z} = \frac{1}{2}$? $S^{z} = -\frac{1}{2}$?

<i>Hint: </i>You can modify the program to accomplish this by adding more conditionals <i>or</i> by clervly removing a conditional. If spins with $S^{x}=\frac{1}{2}$ and $S^{x}=-\frac{1}{2}$ are to be analyzed again, do you need to check for the value of $\hat{S}^{x}$?

<b>Answer: </b> Total: 25000. About 12500 spins had $S^{z}=\frac{1}{2}$, but this only includes those with $S^{x}=\frac{1}{2}$. Also analyzing those with $S^{x} = -\frac{1}{2}$ (about 25000 of these), about half (another 12500) will have $S^{z} = \frac{1}{2}$. This doubles the result from the previous question, giving about 25000.

As a shortcut, since we're going to re-measure $\hat{S}^{z}$ regardless of the value of $S^{x}$, we can just take out the conditional which selects only $S^{x}>0$. We can also stop keeping track of how many have $S^{x}=\pm\frac{1}{2}$.

In [218]:
N = 100000

Nup = 0
Ndown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        s2,state2 = measure_op(state,sx,2)
        s3,state3 = measure_op(state2,sz,2)
        if s3>0:
            Nup2 = Nup2 + 1
        else:
            Ndown2 = Ndown2 + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('Of the ' + str(Nup) + ' |+> states, a second measurement revealed ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')


The beam produced 50198 |+> states and 49802 |-> states.
Of the 50198 |+> states, a second measurement revealed 25172 |+> states and 25026 |-> states.


(6) Now let's stop filtering with respect to $\hat{S}^{z}$ at the very beginning. We will still measure it, but instead of only keeping those with $S^{z}=\frac{1}{2}$, we'll pass all states into the measurement of $\hat{S}^{x}$ (and then another measurement of $\hat{S}^{z}$. How many will end up with $S^{z}=\frac{1}{2}$?

Does your answer suggest that the "quantum weirdness" is resolved? That is, in this setup, if a spin is initially determined to have $S^{z}=\frac{1}{2}$, will the second measurement of $\hat{S}^{z}$ (which takes place <i>after</i> $\hat{S}^{x}$ has been measured) also return $S^{z} = \frac{1}{2}$?

<b>Answer: </b> About 50000. Note that this won't be <i>exactly</i> the same as the number of $\left|+\right\rangle$ states found in the initial measurement. The measurement of $\hat{S}^{x}$ will still scramble $\hat{S}^{z}$ so that if you were to track an individual spin which (for example) initially gave $S^{z} = \frac{1}{2}$, a second measurement could be $\frac{1}{2}$ or $-\frac{1}{2}$ (with equal probability). The quantum weirdness is not resolved but buried in statistics. Unless you find <i>exactly</i> the same distribution of $\left|+\right\rangle$ and $\left|-\right\rangle$ in the output of each measurement of $\hat{S}^{z}$, some of the spins <i>have</i> to "change" values.

In [220]:
N = 100000

Nup = 0
Ndown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        s2,state2 = measure_op(state,sx,2)
        s3,state3 = measure_op(state2,sz,2)
        if s3>0:
            Nup2 = Nup2 + 1
        else:
            Ndown2 = Ndown2 + 1
    else:
        Ndown = Ndown +1
        s2,state2 = measure_op(state,sx,2)
        s3,state3 = measure_op(state2,sz,2)
        if s3>0:
            Nup2 = Nup2 + 1
        else:
            Ndown2 = Ndown2 + 1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('After measuring Sx, we find ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')

The beam produced 50141 |+> states and 49859 |-> states.
After measuring Sx, we find 49759 |+> states and 50241 |-> states.


# STERN-GERLACH EXPERIMENT (BEAM RECOMBINATION)

The McIntyre text discusses a modification of the experiment presented above in which $\hat{S}^{x}$ is still <i>analyzed</i>, but the beams are not split. This is subtly different from what occurs in the last exercises in which we obtained measurement information but did not <i>store</i> it. As we'll see, forcing the quantum state into a well-defined value of (say) $\hat{S}^{x}$ is what has such spooky consequences for the notion of the operator $\hat{S}^{z}$ having a "well-defined" value.

What we would like to do in this last exploration is to analyze $\hat{S}^{x}$ after peeling off the subset of unpolarized spins with $S^{z} = \frac{1}{2}$ without actually measuring $\hat{S}^{x}$. To accomplish this, we will modify the <b>measure_op()</b> so that while it does split the state <b>v</b> into a linear superposition of $\left|+\right\rangle_{x}$ and $\left|-\right\rangle_{x}$, it does not <i>project</i> the state onto one of these eigenstates through a measurement that returns either $S^{x} = \frac{1}{2}$ or $S^{x} = -\frac{1}{2}$.

To begin, make a copy of the function definition for <b>measure_op()</b> and rename it <b>analyze_op()</b>, leaving the input arguments unchanged.

In [ ]:
def analyze_op(v,op,n):
    #operator
    w,u = eigh(op)
    
    #overlap coefficients
    #c = zeros(n,dtype='complex')
    
    for i in range(0,n):
        c = squeeze(asarray(dot(v,conj(u))))
        
    #measure by selecting random number
    r = rand()
    
    csum = abs(c[0])**2
    nf = 0
    
    while (r>csum):
        nf = nf + 1
        csum = csum + abs(c[nf])**2
        
    vf = squeeze(asarray(u[:,nf]))
    val = w[nf]
    
    return val, vf

Remove everything from (and including) the line "<b>r = rand()</b>" to (and including) "<b>val = w[nf]</b>." This section performs the actual measurement and projection of the input state onto the corresponding eigenstate of the operator in question.

Take out <b>val</b> from the return command so it just reads "<b>return vf</b>." This will ensure that the function only returns the state after being analyzed. The remainder of the code writes the input state as a combination of eigenstates of the operator being measured (or now, analyzed). The last step is to construct <b>vf</b> in terms of the eigenstates of the operator. The following line inserted before <b>return vf</b> will do the job:

<b>vf = squeeze(asarray(dot(c,u)))</b>

In [248]:
def analyze_op(v,op,n):
    #operator
    w,u = eigh(op)
    
    #overlap coefficients
    #c = zeros(n,dtype='complex')
    
    for i in range(0,n):
        c = squeeze(asarray(dot(v,conj(u))))
        
    vf = squeeze(asarray(dot(c,u)))
    
    return vf

Now replace <b>measure_op(...,sx,...)</b> by <b>analyze_op(...,sx,...)</b> in the Stern-Gerlach experiment which takes only the states initially measured to be $\left|+\right\rangle$, measures $\hat{S}^{x}$ (now it will only <i>analyze</i> this), and then measures $\hat{S}^{z}$ again. This process is equivalent to Experiment 4(c) in the text in which the beams are recombined after passing through an analyzer of $\hat{S}^{x}$ without actually performing a measurement. Of the initial number of spins with $S^{z} = \frac{1}{2}$, how many do you expect to return a value $S^{z} = -\frac{1}{2}$?

In [251]:
N = 100000

Nup = 0
Ndown = 0

Nup2 = 0
Ndown2 = 0

for i in range(0,N):
    v = newstate(2)
    s,state = measure_op(v,sz,2)
    
    if s>0:
        Nup = Nup + 1
        state2 = analyze_op(state,sx,2)
        s3,state3 = measure_op(state2,sz,2)
        if s3>0:
            Nup2 = Nup2 + 1
        else:
            Ndown2 = Ndown2 + 1
    else:
        Ndown = Ndown +1
        
print('The beam produced ' + str(Nup) + ' |+> states and ' + str(Ndown) + ' |-> states.')
print('Of the ' + str(Nup) + ' |+> states, a second measurement revealed ' + str(Nup2) + ' |+> states and ' + str(Ndown2) + ' |-> states.')


The beam produced 50015 |+> states and 49985 |-> states.
Of the 50015 |+> states, a second measurement revealed 50015 |+> states and 0 |-> states.
